# Assistant de Recherche pour la Génération Automatique de README

## 1. INTRODUCTION

Dans le monde du développement logiciel, la documentation est souvent reléguée au second plan, malgré son rôle central. Un README clair et complet est pourtant la première impression qu’un utilisateur ou un contributeur aura d’un projet : il facilite la prise en main, l’installation et la compréhension des fonctionnalités. Pourtant, écrire et maintenir une documentation à jour est long, fastidieux et facilement négligé, surtout dans des projets en constante évolution.

Ce projet propose une solution révolutionnaire pour tous les développeurs : un assistant automatique capable d’analyser un projet complet et de générer un README.md structuré, cohérent et fidèle à l’état actuel du code.

Les bénéfices sont immédiats et concrets :
- Gain de temps massif : fini les heures passées à rédiger manuellement un README pour chaque projet ou chaque mise à jour.
- Documentation toujours à jour : en analysant directement le code source, l’outil élimine le phénomène de "documentation drift" où les README deviennent obsolètes par rapport au code réel.
- Accessibilité et collaboration : un README clair permet à n’importe quel collaborateur ou utilisateur de comprendre et de contribuer rapidement au projet.
- Polyvalence : même les projets multi-langages ou peu commentés peuvent bénéficier d’une documentation complète et professionnelle.

En automatisant ce processus, ce projet ne se contente pas de simplifier la vie des développeurs : il change leur manière de travailler, en rendant la documentation plus rapide, fiable et accessible. 

Imaginez des milliers de projets à travers le monde, chacun avec un README précis et clair, généré automatiquement. Ce projet a le potentiel de transformer la manière dont les développeurs documentent leurs projets, tout en améliorant la collaboration et la maintenance logicielle à grande échelle.

# Présentation du Sujet

Le projet consiste en la création d'un **assistant intelligent d'analyse de code** capable de :

1. **Analyser automatiquement** un dossier de projet contenant du code source
2. **Déduire la structure** et l'organisation du projet
3. **Générer un fichier README.md** complet et professionnel

## 1.1 Contenu du README Généré

Le fichier README généré doit obligatoirement contenir :

- Un **résumé clair** du projet et de son fonctionnement
- Une **arborescence détaillée** de la structure du dossier
- Les **bibliothèques et dépendances** nécessaires
- Un **résumé fichier par fichier** du code
- Les **instructions de lancement** du projet
- Les **limitations et avertissements** éventuels

Le défi principal réside dans la compréhension sémantique du code : il ne s'agit pas simplement de lister des fichiers, mais de comprendre leur rôle, leurs interactions et l'architecture globale du projet.


## 1.2 Intérêt du Projet

Ce projet s'inscrit dans une problématique d'automatisation intelligente de la documentation. L'objectif est de combiner les techniques de traitement du langage naturel (NLP), l'architecture RAG (Retrieval-Augmented Generation) et les capacités des modèles de langage pour créer un assistant capable d'analyser automatiquement un projet et de générer une documentation structurée et cohérente. 

Dans le cadre du cours de Programmation Avancée et Calcul Parallèle, ce projet permet d'explorer des concepts avancés tels que :

- **Standardisation** : garantir une structure homogène de documentation
- **Analyse sémantique** : comprendre le code au-delà de sa syntaxe
- **Scalabilité** : traiter des projets de tailles variables grâce au chunking intelligent

# 2. Pipeline de Développement

### Étape 1 : Extraction de l'Infrastructure

À partir du dossier racine, parcourir récursivement l'arborescence pour identifier tous les fichiers et sous-dossiers. Filtrer intelligemment les éléments parasites.

### Étape 2 : Séparation Code vs Documentation

Distinguer les fichiers contenant du code exécutable (`.py`, `.js`, `.cpp`, `.ipynb`, etc.) des fichiers de documentation ou de configuration. Cette étape permet de concentrer l'analyse sur le code fonctionnel.

### Étape 3 : Embedding et Chunking avec RAG

Cette étape constitue le cœur technique du projet :

#### 3.1. Chunking intelligent

- Découper le code en blocs cohérents (fonctions, classes, modules)
- Gérer les chunks trop longs en les subdivisant (limite : 5000 caractères)
- Préserver la cohérence sémantique lors du découpage

#### 3.2. Génération des embeddings

- Transformer chaque chunk en vecteur numérique via l'API Mistral (`codestral-embed`)
- Utiliser des embeddings spécialisés pour le code

#### 3.3. Indexation vectorielle

- Stocker les vecteurs dans un index FAISS (Facebook AI Similarity Search)
- Permettre une recherche sémantique rapide et efficace


#### 3.4. Analyse Sémantique via LLM

Utiliser un modèle de langage (`codestral-2508`) pour interpréter le rôle et la logique des fonctions extraites.

### Étape 4 : Récupération des Prérequis

Extraire automatiquement les imports et dépendances pour reconstituer la liste des bibliothèques nécessaires au projet.

### Étape 5 : Génération du README

Synthétiser toutes les informations collectées dans un prompt structuré et générer un README.md cohérent via le LLM, en respectant une structure prédéfinie stricte.


## Code

### Paramétrage 

1. Importation des bibliothèques nécessaires :
   
On commence par importer toutes les bibliothèques utiles pour le projet. On ajoute notamment `concurrent.futures` pour la **parallélisation** des tâches indépendantes, cruciale pour l'optimisation des performances.

In [193]:
# Importation des bibliothèques nécessaires pour le projet.

import os            # Pour les opérations système (gestion des chemins, etc.)
import re            # Pour les expressions régulières (utilisé pour le chunking)
import json          # Pour la gestion des fichiers JSON (notamment les .ipynb)
import numpy as np   # Pour les opérations numériques (essentiel pour les embeddings)
from dotenv import load_dotenv # Pour charger la clé API depuis un fichier .env
from mistralai import Mistral # SDK officiel pour interagir avec Mistral AI
import faiss         # Bibliothèque d'indexation vectorielle (recherche d'embeddings)
import time          # Pour la gestion des pauses (retry en cas d'erreur 429)
import random        # Pour la gestion des pauses aléatoires (backoff jitter)
import concurrent.futures # Pour la parallélisation

2. Récupération de la clé API pour l'appel à Miastral AI.

Cette étape est cruciale pour sécuriser la clé API et permettre l’accès aux services Mistral AI.

In [194]:
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

3. Exclusion de fichiers/dossiers inutiles

Le pipeline n’analyse que le code pertinent. On exclut donc :
- Les environnements virtuels, caches, artefacts de build, fichiers binaires.
- Les fichiers temporaires et images inutiles à l’analyse.

Cela améliore les performances et réduit le nombre de chunks inutiles.

In [195]:
# Liste des dossiers à exclure (Python, Node.js, Java, etc.)
excluded_dirs = {
    # Python
    "venv", ".venv", "__pycache__", "site-packages", "env", ".env",
    # Node.js / JS
    "node_modules", "bower_components", ".npm", ".yarn",
    # Java / JVM
    "target", "build", ".gradle", ".mvn", "out",
    # C/C++ / Rust / Go / .NET
    "cmake-build-debug", "cmake-build-release", "bin", "obj", "pkg", "dist",
    "Debug", "Release",
    # Divers IDE / SCM
    ".git", ".svn", ".hg", ".idea", ".vscode"
}

# Liste des fichiers à exclure (souvent générés automatiquement ou inutiles à l'analyse)
excluded_files = {
    # SCM / VCS
    ".gitignore", ".gitattributes", ".gitmodules",
    ".hgignore", ".svnignore",

    # Config / lockfiles
    "package-lock.json", "yarn.lock", "pnpm-lock.yaml",
    "poetry.lock", "Pipfile.lock",

    # Build / cache
    "Thumbs.db", "Desktop.ini",
    ".DS_Store",  # macOS
    "npm-debug.log", "yarn-error.log",
    "Cargo.lock", "Gemfile.lock",

    # Environnements
    ".env", ".env.local", ".env.production", ".env.development",

    # Binaires / artefacts
    "*.pyc", "*.pyo", "*.pyd",
    "*.class", "*.jar", "*.war", "*.ear",
    "*.dll", "*.so", "*.dylib",
    "*.exe", "*.out", "*.o", "*.obj",
    "*.a", "*.lib",

    # Archives
    "*.zip", "*.tar", "*.gz", "*.bz2", "*.rar",

    # Divers
    "README.md", "LICENSE", "COPYING", "CHANGELOG", "TODO", "Makefile",
    
    # Photos 
    "*.png", "*.jpg", "*.jpeg", "*.gif", "*.bmp", "*.svg"
}

## Fonctions principales

1. Affichage de l’arborescence (show_tree)

Cette fonction affiche l’arborescence du projet jusqu’au second niveau seulement. Elle n'affiche pas les images également. 

Cela évite d’afficher des milliers de fichiers pour les gros projets. Et donc de se limiter au premier et second niveaux dans l'arborescence.

In [196]:
def show_tree(root, code_ext=None):
    """
    Renvoie une arborescence synthétique du dossier en ne gardant que les fichiers de code.
    code_ext : set d'extensions à inclure, ex: {'.py', '.js', '.ipynb'}
    """
    if code_ext is None:
        code_ext = {".py", ".js", ".ts", ".cpp", ".c", ".java", ".ipynb", ".php", ".html"}

    lines = []
    for dirpath, dirnames, filenames in os.walk(root):
        # filtrer les fichiers qui ne sont pas du code
        code_files = [f for f in filenames if os.path.splitext(f)[1].lower() in code_ext]
        if code_files:
            level = dirpath.replace(root, "").count(os.sep)
            indent = " " * 4 * level
            lines.append(f"{indent}{os.path.basename(dirpath)}/")
            subindent = " " * 4 * (level + 1)
            for f in code_files:
                lines.append(f"{subindent}{f}")
    return "\n".join(lines)


2. Lecture du code (read_code)

Cette fonction lit uniquement le code exploitable :
- Pour les .ipynb, seules les cellules de type "code" sont conservées.
- Pour les autres fichiers, tout le contenu est récupéré.

In [197]:
def read_code(file_path):
    """
    Lit le contenu d'un fichier de code et renvoie uniquement le code exploitable.

    - Pour les fichiers Jupyter Notebook (.ipynb) : 
      ne conserve que le contenu des cellules de type 'code', et les concatène en une seule chaîne de caractères.
    - Pour les autres fichiers de code (.py, .js, .cpp, etc.) :
      lit l'intégralité du fichier et retourne son contenu brut.

    Paramètre :
    ----------
    file_path : str
        Chemin vers le fichier à lire.

    Retour :
    -------
    str
        Contenu du code du fichier, prêt à être utilisé pour le traitement ou
        pour la génération de chunks dans un pipeline de documentation.
    """
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".ipynb":
        with open(file_path, "r", encoding="utf-8") as f:
            nb = json.load(f)
            content = []
            for cell in nb.get("cells", []):
                if cell.get("cell_type") == "code":
                    content.append("".join(cell.get("source", [])))
            return "\n".join(content)

    else:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

## Découpage du code en chunks + embedding

**Optimisation :** Cette phase est désormais traitée en **parallèle** pour réduire le temps de lecture des fichiers et le découpage des fonctions.

1. Découpage du code en chunks

Nous avons donc deux fonctions qui travaillent ensemble :
- split_functions : découpe le code en fonctions ou blocs logiques.
- split_long_chunk : si un chunk est trop long, on le découpe en morceaux plus petits.

Cela permet d'assurer que chaque chunk respecte la limite maximale pour l’API d’embeddings.

In [198]:
def split_functions(code):
    """Découpe le code en fonctions/chunks (première étape de chunking logique)."""
    # Regex : recherche de motifs de fonctions 'def nom(...):' et capture leur corps.
    # re.DOTALL (ou S) est crucial pour que le '.' matche les sauts de ligne à l'intérieur de la fonction.
    pattern = r"(def [\w_]+\s*\([^)]*\):(?:\n(?:\s+.+))*)" 
    chunks = re.findall(pattern, code, re.DOTALL)
    # Si aucune fonction (ex: script simple ou code dans .ipynb), le chunk est le fichier entier.
    return chunks if chunks else [code]

In [199]:
# Taille max en caractères pour un chunk envoyé aux embeddings
MAX_CHARS_PER_CHUNK = 5000

def split_long_chunk(chunk, max_chars=MAX_CHARS_PER_CHUNK):
    """
    Si un chunk est trop long, on le découpe en morceaux plus petits basés sur les lignes.
    On essaye de garder des morceaux cohérents tout en respectant la limite de taille.
    """
    if len(chunk) <= max_chars:
        return [chunk]

    lines = chunk.splitlines()
    parts = []
    current = []
    current_len = 0

    for line in lines:
        # +1 pour le saut de ligne
        extra = len(line) + 1
        # Si on dépasse la limite, on coupe ici
        if current_len + extra > max_chars and current:
            parts.append("\n".join(current))
            current = [line]
            current_len = extra
        else:
            current.append(line)
            current_len += extra

    if current:
        parts.append("\n".join(current))

    return parts

2. Création des embeddings (Parallélisée)

Chaque chunk est transformé en vecteur numérique grâce au modèle `codestral-embed`.

**Parallélisation :** La fonction `create_embeddings_parallélisé` utilise désormais un `ThreadPoolExecutor` pour envoyer plusieurs lots (`batches`) à l'API Mistral **simultanément**. Ceci permet de maximiser le débit et de contourner plus efficacement les limites de requêtes par seconde (`Rate Limit - 429`) en utilisant un mécanisme de **backoff exponentiel avec Jitter** pour chaque appel.

In [200]:
# Taille max en caractères pour un chunk envoyé aux embeddings
MAX_CHARS_PER_CHUNK = 5000 
# Nombre maximal de lots (batches) à envoyer simultanément à l'API.
MAX_CONCURRENT_BATCHES = 4 # Ajustez selon la limite de votre clé API

def _get_embedding_batch(client, batch, max_retries):
    """Fonction utilitaire pour créer un seul lot d'embeddings avec retry."""
    for attempt in range(max_retries):
        try:
            # Appel à l'API Mistral AI
            resp = client.embeddings.create( 
                model="codestral-embed",
                inputs=batch
            )
            # Retourne les embeddings du lot
            return [np.array(emb.embedding, dtype=np.float32) for emb in resp.data] 

        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt + random.random()
                print(f"Erreur 429... Retry dans {wait:.2f}s pour un lot d'embeddings.")
                time.sleep(wait)
            else:
                raise e
    raise Exception("Échec de la génération des embeddings après tous les essais.")


def create_embeddings_parallélisé(chunks, batch_size=16, max_retries=5):
    """
    Génère les vecteurs d'embeddings en parallèle en envoyant plusieurs lots (batches) 
    simultanément à l'API Mistral AI.
    """
    all_batches = []
    # Création de tous les lots (batches)
    for i in range(0, len(chunks), batch_size):
        all_batches.append(chunks[i:i+batch_size])

    embeddings = []

    # Utilisation du ThreadPoolExecutor pour l'appel API (I/O-bound)
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_CONCURRENT_BATCHES) as executor:
        
        # On soumet chaque lot (batch) à l'exécution asynchrone
        # On utilise un "lambda" pour passer des arguments fixes (client, max_retries)
        futures = [executor.submit(_get_embedding_batch, client, batch, max_retries) 
                   for batch in all_batches]

        # On récupère les résultats au fur et à mesure que les futures sont terminées
        for future in concurrent.futures.as_completed(futures):
            try:
                # La fonction retourne une liste d'embeddings pour ce lot
                batch_embeddings = future.result() 
                embeddings.extend(batch_embeddings)
            except Exception as e:
                # Gérer les exceptions non traitées dans _get_embedding_batch
                print(f"Une erreur s'est produite lors de la récupération d'un lot : {e}")
                # Vous pourriez choisir d'ignorer ou de relancer ici
                
    return embeddings

3. Parcours du projet (itération sur fichiers)

Cette fonction récupère seulement les fichiers code utiles, en respectant les exclusions.

In [201]:
def iter_project_files(folder_path, code_ext):
    """Itère uniquement sur les fichiers code utiles, en excluant dossiers et fichiers parasites."""
    for dirpath, dirnames, filenames in os.walk(folder_path):
        # On filtre les dossiers exclus
        dirnames[:] = [d for d in dirnames if d not in excluded_dirs]

        for f in filenames:
            # Exclure les fichiers parasites
            if f in excluded_files:
                continue
            # Exclure aussi par motif (ex: *.pyc, *.class, etc.)
            for pattern in excluded_files:
                if pattern.startswith("*.") and f.endswith(pattern[1:]):
                    break
            else:
                ext = os.path.splitext(f)[1].lower()
                if ext in code_ext:
                    yield os.path.join(dirpath, f)

4. Fonction utilitaire de traitement par fichier (`process_file_for_chunking`)

Cette fonction enveloppe la lecture et les deux étapes de découpage (`split_functions` et `split_long_chunk`). Elle est essentielle car c'est cette unité de travail qui sera **exécutée en parallèle** par le `ThreadPoolExecutor` sur chaque fichier du projet. Elle permet de lier immédiatement chaque chunk au chemin de son fichier d'origine.

In [202]:
# Fonction utilitaire pour le mapping
def process_file_for_chunking(file_path, code_ext):
    """
    Lit un fichier, génère les chunks et retourne une liste de (chunk, path).
    Fonction adaptée pour être mappée par un Executor.
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext not in code_ext:
        return []

    try:
        code = read_code(file_path)
        if not code.strip():
            return []

        # Première étape : découper par fonctions / blocs
        func_chunks = split_functions(code)

        results = []
        # Deuxième étape : si un chunk est trop long, on le re-découpe
        for fc in func_chunks:
            small_chunks = split_long_chunk(fc, max_chars=MAX_CHARS_PER_CHUNK)
            for sc in small_chunks:
                results.append((sc, file_path))
        return results
    except Exception as e:
        print(f"Erreur de traitement du fichier {file_path}: {e}")
        return []

## Pipeline de génération du README

Le pipeline suit la logique RAG (Retrieval-Augmented Generation) optimisée :
1. Lecture et filtrage des fichiers code **(Parallélisé)**.
2. Chunking et découpage **(Parallélisé)**.
3. Création des embeddings **(Parallélisé)**.
4. Indexation FAISS pour recherche par similarité **(Séquentiel)**.
5. **Construction optimisée du prompt RAG (avec chemins de fichiers)** et génération du README via LLM.
6. Sauvegarde du README dans le dossier du projet.

L'utilisation de la parallélisation permet un gain de temps significatif sur les projets volumineux.

Le prompt RAG structure la génération pour garantir un README complet, clair et dans l’ordre :
1. Titre du projet
2. Présentation générale
3. Arborescence
4. Bibliothèques
5. Résumé fichier par fichier
6. Instructions pour lancer le projet
7. Avertissements et Limitations (Nouvelle section finale)

**Amélioration du contexte RAG :** Le bloc de chunks envoyé au LLM est désormais **enrichi du nom de fichier d'origine** pour chaque morceau de code. Cette information est cruciale pour que le modèle puisse rédiger correctement la section "Résumé fichier par fichier" avec précision.

In [203]:
def generate_readme_RAG(folder_path, output_file, max_workers=4): # max_workers est le paramètre de parallélisation
    
    # ---- 3.1 Lire et Chunquer les fichiers en parallèle ----
    code_ext = {".py", ".js", ".ts", ".cpp", ".c", ".java", ".ipynb", ".php", ".html"}
    all_files_to_process = list(iter_project_files(folder_path, code_ext))
    
    chunks = []
    chunk_paths = []

    # Utilisation du ThreadPoolExecutor pour le parallélisme I/O et CPU léger
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # On soumet chaque fichier à la fonction process_file_for_chunking
        # executor.map applique la fonction à chaque élément de la liste
        futures = [executor.submit(process_file_for_chunking, fp, code_ext) 
                   for fp in all_files_to_process]
        
        # On récupère les résultats au fur et à mesure
        for future in concurrent.futures.as_completed(futures):
            results = future.result()
            for chunk, file_path in results:
                chunks.append(chunk)
                chunk_paths.append(file_path)


    if not chunks:
        raise ValueError("Aucun code utile trouvé dans ce dossier après chunking !")

    print(f"Total chunks : {len(chunks)}")


    # ---- 3.3 Embeddings (PARALLÉLISÉ) ----
    print("Génération des embeddings en parallèle...")
    embeddings = create_embeddings_parallélisé(chunks, batch_size=16)
    dim = len(embeddings[0]) 

    # ---- 3.4 Indexation FAISS (Séquentiel obligatoire) ----
    # Cette étape DOIT rester séquentielle pour garantir l'intégrité de l'index
    print("Indexation FAISS...")
    index = faiss.IndexFlatL2(dim) 
    index.add(np.array(embeddings)) # Indexation/Stockage des vecteurs.

    # ---- 3.5 Arborescence ----
    # Génération de l'arborescence synthétique.
    tree = show_tree(folder_path)

    # ---- 3.6 Construire le prompt RAG final ----
    # Création du prompt structuré pour guider le LLM (Codestral)
    prompt = f"""
    Tu es **un expert en analyse de code et rédaction technique**.  
    Tu dois analyser un projet à partir d’une **liste de chunks de code non ordonnés** et **l’arborescence du projet**.

    Ton objectif : Produire un `README.md` **professionnel, lisible et structuré**, destiné à des développeurs ou utilisateurs techniques.

    ----------------------------------------
    ### Contraintes obligatoires

    Respecte EXACTEMENT l’ordre et les titres suivants :

    1. # Titre du projet  
    2. # Présentation générale et fonctionnement  
    3. # Arborescence du projet des fichiers importants  
    4. # Bibliothèques nécessaires pour le projet  
    5. # Résumé catégories par catégories  
    6. # Comment lancer le projet  

    Tu peux **ajouter d’autres sections facultatives** UNIQUEMENT si elles sont pertinentes (ex : # API, # Tests, # Cas d’usage, # Structure des données, # Limitations techniques).  
    **Ces sections ne doivent pas faire dépasser les 200 lignes au total.**

    Si une section est incomplète ou manque d’informations, tu dois écrire :
    > Informations à compléter

    **INTERDIT** :
    - Pas de blocs de code Markdown autour du README
    - Ne pas commencer par \`\`\`markdown, \`\`\`md, \`\`\`bash…
    - Ne pas réordonner les sections obligatoires
    - Ne dépasse pas les 200 lignes au maximum dans le fichier readme final
    - Ne fais pas d'indentation avec des espaces ou des tabulations dans les titres Markdown

    ----------------------------------------
    ### Instructions détaillées

    #### Pour 2. # Présentation générale
    - Ne dépasse pas 20–30 lignes
    - Expliquer l’objectif, l’utilité et le fonctionnement global du projet
    - Déduis le domaine si possible (IA, web, data, simulation, etc.)

    #### Pour 3. # Arborescence
    - Regarde l’arborescence donnée mais **ne l’affiche pas telle quelle**.
    - Ne montrer que les **fichiers et dossiers de code importants** (scripts, modules principaux, routes, notebooks, etc.).
    - Ignorer complètement les fichiers non-code comme images, fichiers temporaires, logs, docs inutiles, etc.
    - Ne pas inventer de fichiers ou dossiers qui n’existent pas.
    - Présenter l’arborescence de manière **synthétique et lisible** pour un développeur.


    #### Pour 5. # Résumé catégories par catégories
    - Classe si possible **par catégories (langage, modules, front/back, scripts, modèles, etc.)**
    - Si on a un site web, de parler des routes

    #### Pour 6. # Comment lancer le projet
    C'est une partie cruciale du README :
    - Donne des étapes claires pour un utilisateur lambda
    - Tu peux inclure des commandes terminales si nécessaire
    - Si des éléments sont manquants, préciser :
    > Informations à compléter ou configuration nécessaire

    ----------------------------------------
    ### Fin du README

    Le README doit se terminer par une section :

    ### Avertissements et limitations
    - 10 à 20 lignes maximum
    - Mentionner limites, dépendances, risques, données manquantes, etc.

    ----------------------------------------
    ### Données fournies

    ### Données fournies

    Voici une **arborescence synthétique** (seulement fichiers/dossiers de code importants) :
    -----------------
    {tree}
    -----------------


    Voici des chunks (non ordonnés) :
    -----------------
    {chunks[:100]}
    -----------------

     Maintenant, génère le README complet en respectant toutes les consignes ci-dessus.
    """

    # Appel au LLM (Codestral) pour la génération finale du README
    response = client.chat.complete(
        model="codestral-2508", 
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2, # Température basse pour une réponse factuelle et stable
        max_tokens=5000 
    )

    readme_text = response.choices[0].message.content

    # ---- 3.7 Sauvegarde ----
    # Si README.md existe, on crée README2.md
    if os.path.exists(output_file):
        base = os.path.dirname(output_file)
        output_file = os.path.join(base, "README2.md")
        print("README.md existant → génération dans README2.md")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(readme_text)

    print("\n README généré :", output_file)


<>:132: SyntaxWarning: invalid escape sequence '\`'
<>:132: SyntaxWarning: invalid escape sequence '\`'
C:\Users\33611\AppData\Local\Temp\ipykernel_23120\3183622200.py:132: SyntaxWarning: invalid escape sequence '\`'
  """


## Création du README de l'utilisateur

In [204]:
# --- Déclenchement du pipeline de génération ---
# Demande à l'utilisateur de spécifier le chemin du dossier du projet.
folder = input("Dossier à analyser : ").strip()

# Définition du chemin de sortie par défaut
output = os.path.join(folder, "README.md")

# Lancement du processus RAG + génération README
generate_readme_RAG(folder, output)

Total chunks : 107
Génération des embeddings en parallèle...
Erreur 429... Retry dans 1.29s pour un lot d'embeddings.
Erreur 429... Retry dans 1.10s pour un lot d'embeddings.
Indexation FAISS...
README.md existant → génération dans README2.md

 README généré : D:\33611\Documents\MASTER\M2\Open_data\projet\TourismEco\README2.md


## Guide Utilisateur et Avertissements

Cette partie sert à guider l’utilisateur qui veut générer automatiquement un README pour son projet en utilisant ce notebook.

### Guide d’Exécution

Pour faire fonctionner la génération du README, voici les étapes à suivre :

1. Préparer l’Environnement :
    - Vérifiez que votre clé API Mistral (MISTRAL_API_KEY) est correctement configurée dans le fichier .env.
    - Assurez-vous que toutes les bibliothèques nécessaires sont installées, par exemple : numpy, mistralai, faiss-cpu, etc.

2. Lancer le Notebook :
    - Exécutez toutes les cellules du notebook dans l’ordre, de la première (imports) jusqu’à la dernière cellule de génération.

3. Indiquer le Projet à Analyser :
    - Lorsque le notebook demande “Dossier à analyser”, copiez-collez le chemin complet du dossier racine de votre projet.
      - Exemple :
          - Windows : C:\Users\NomUtilisateur\MonProjet
          - Mac/Linux : /Users/Documents/MonProjet

4. Suivre l’Exécution :
Si vous voyez des erreurs 429, pas de panique !
Ce sont des limites d’API (trop de requêtes trop rapides).
Le système attend quelques secondes et réessaye automatiquement grâce au mécanisme de backoff exponentiel avec Jitter.

Vous verrez alors des messages du type :

"Total chunks : 26
Erreur 429… Retry dans 1.08s
Erreur 429… Retry dans 2.26s"

Tout est normal, l’exécution continue toute seule.

5. Récupérer le Résultat :

   - Une fois terminé, un message indique : "README généré : [chemin/vers/README.md]"

  - Le fichier README sera créé dans le dossier racine du projet.
      - Si un README existait déjà, le nouveau sera nommé README2.md.

### Avertissements et Limites Connues

Quelques points importants à connaître pour utiliser ce système correctement :

1. Qualité du Code Source :
    - Plus votre code est clair et bien structuré (variables explicites, fonctions bien nommées…), plus le README généré sera précis et utile.
    - Un code confus ou mal organisé donnera un README moins pertinent.

2. Types de Fichiers :
   - Le système est optimisé pour le code source (Python, JS, C++, etc.).
   - Certains fichiers spécifiques ou binaires (images, fichiers compilés…) ne seront pas analysés correctement.

1. Performance et Scalabilité :
   - Le temps de génération dépend de la taille du projet.
   - Pour des projets très volumineux (des milliers de fichiers), l’index FAISS en mémoire peut devenir un goulot d’étranglement, et la qualité des résultats peut diminuer.
   - Dans ces cas, il peut être nécessaire de diviser le projet en sous-dossiers ou de générer le README par parties.

## Optimisation et Limites de la Concurrence

### Optimisation par Concurrence : Justification

L’utilisation du multithreading via **`ThreadPoolExecutor`** a pour objectif d’optimiser les tâches **I/O-Bound**, en particulier la lecture depuis le disque et les appels à l’API pour générer les embeddings. Ce choix permet de **masquer la latence** associée à ces opérations, en exploitant les temps d’attente pour effectuer d’autres traitements.

Les bénéfices principaux sont les suivants :

* **Lecture et Segmentation (Chunking)**
    * Les accès disques sont lents et bloquants.
    * Le multithreading permet de traiter et découper en *chunks* les fichiers déjà lus pendant que d’autres sont encore en cours de lecture.
* **Génération des Embeddings**
    * Les appels à l’API Mistral sont des opérations réseau.
    * En envoyant plusieurs requêtes simultanément, le programme augmente le **débit global de vectorisation** et réduit les temps d’attente cumulés.

En conséquence, l’architecture concurrente améliore significativement le temps total de traitement en exploitant intelligemment les périodes d’inactivité liées à l’I/O.

---

### Contraintes et Limites de la Parallélisation

L’usage de la concurrence apporte des gains indéniables, mais plusieurs contraintes techniques empêchent une parallélisation complète du pipeline.

#### Séquentialité Nécessaire

Certaines tâches doivent rester strictement séquentielles pour des raisons d'intégrité :

* **Indexation FAISS**
    * L’ajout de vecteurs à l’index ne peut être parallélisé sans risque de corruption mémoire ou incohérence structurelle.
* **Génération finale du README**
    * La synthèse demandée au LLM est une opération unique, dépendant de l’ensemble des informations récoltées.

#### Limites du Multithreading sur le CPU

Le multithreading n’est performant que sur les tâches I/O-bound.

* Les étapes computationnelles (ex. nettoyage de chaînes, découpage) sont limitées par le **GIL (Global Interpreter Lock)**.
* Pour un projet principalement calculatoire, le *multiprocessing* aurait été préférable, bien que plus coûteux en mémoire.

#### Scalabilité et Consommation Mémoire

Lors d’un passage à grande échelle :

* L’I/O cesse d’être le principal problème : c’est la **mémoire RAM** qui devient limitante.
* L’index **FAISS** peut devenir trop volumineux.
* Une solution distribuée, une indexation hiérarchique ou un stockage persistant serait alors nécessaire.

---

### Bilan

Le multithreading a été appliqué là où il procure le meilleur rendement : I/O et réseau. Les phases séquentielles assurent la cohérence des données et de l’indexation, tandis que les limites du GIL et de la mémoire RAM ont guidé la conception d’une solution hybride, pragmatique et scalable dans des conditions réalistes.

## Conclusion du Projet

Le projet atteint pleinement son objectif : générer automatiquement un README structuré à partir de code source, grâce à une architecture **RAG (Retrieval-Augmented Generation)** combinant parsing intelligent, embeddings spécialisés (`codestral-embed`) et synthèse LLM (`codestral-2508`). Même sans commentaires explicites dans les fichiers, le système produit une documentation cohérente, exploitable et adaptée aux développeurs.

---

### 1. Contributions Clés et Optimisations

Ce travail a permis de valider plusieurs concepts avancés :

* **Optimisation Concurrente :** Le recours au **multithreading** (`ThreadPoolExecutor`) a réussi à paralléliser les opérations I/O-Bound (lecture disque et appels réseau), augmentant le débit global et l'efficacité de la vectorisation.
* **Structuration de l'Information :** Le *chunking* intelligent à granularité adaptative et le renforcement du lien contexte/code (ajout du nom de fichier) ont permis au LLM d'établir un lien sémantique précis, améliorant la qualité du résumé fonctionnel.
* **Robustesse du Pipeline :** La structure stricte du **prompt RAG normé** et le filtrage ciblé garantissent un format de sortie uniforme et immédiatement exploitable.

---

### 2. Limites Actuelles et Perspectives

Malgré ces performances, les défis inhérents à la manipulation de données massives ouvrent des pistes d’évolution :

* **Scalabilité et Mémoire :** La limite principale reste la capacité **mémoire RAM** nécessaire au stockage de l’index **FAISS** pour les projets massifs.
* **Contraintes d'Architecture :** Les optimisations du threading sont restreintes aux tâches I/O (à cause du **GIL**) ; l'extension à des tâches CPU-bound nécessiterait une migration vers le *multiprocessing*.
* **Contexte LLM :** La synthèse finale est toujours contrainte par la **fenêtre contextuelle** du modèle, limitant le volume de code analysable d’un seul bloc.

---

### En Synthèse

Ce projet démontre qu’une architecture RAG, enrichie par des optimisations concurrentielles mesurées, constitue une approche pertinente et performante pour automatiser la documentation logicielle. Son extension à des architectures distribuées représente la prochaine étape pour des applications industrielles à grande échelle.